**This workbook demonstrates the application of a simple Gamma distribution to evaluating molar fractions and average MW's over a range of defined LMW's, using an analytical form of the integrated Gamma and mass weighted Gamma distribution

In [1]:
import scipy as scipy
from scipy.special import gamma
import math

In [2]:
# Incomplete gamma function as evaluated by Wolfram - differs from native scipy gammainc(a,x)
# Test is incomp_gamma(0.01,0.1) = 1.803241356902546, which is same as https://www.wolframalpha.com/input/?i=gamma(0.01,0.1)
def incomp_gamma(a,x):
    return gamma(a)*(1 - scipy.special.gammainc(a,x))

# Following equation references from "C7+ Characterization of Related Equilibrium Fluids Using the Gamma Distribution"
# By Whitson, Anderson and Soreide

# Calculating the probability and cumulative probability density function
# Probability density function - Eq 1.
def pM(M, eta, alpha, beta):
    return (M - eta)**(alpha-1)*math.exp(-(M-eta)/beta)/(beta**alpha*gamma(alpha))

def Cum_pM(LMW, eta, alpha, plus_MW):
    # Returns analytcally integrated the pM fuction between eta on the lower side, to M on the high side
    # https://www.wolframalpha.com/input/?i=integrate+(x-a)%5Eb*exp(-(x-a)%2Fc)%2Fd
    beta = (plus_MW - eta)/alpha
    
    if LMW <= eta:
        return 0
    a = eta
    b = alpha-1
    c = beta
    d = beta**alpha*gamma(alpha)
    return 1-c*(LMW-a)**b*((LMW-a)/c)**-b*incomp_gamma(b+1,(LMW-a)/c)/d

# And now the mass weighted probability and cumulative mass weighted probability density functions 

# Mass weighted Probability density function.
def pMM(M, eta, alpha, beta):
    return M*(M - eta)**(alpha-1)*math.exp(-(M-eta)/beta)/(beta**alpha*gamma(alpha))

def Cum_pMM(LMW, eta, alpha, plus_MW):
    # Analytically integrates the pMM fuction between eta on the lower side, to LMW on the high side
    #https://www.wolframalpha.com/input/?i=integrate+(x-a)%5Eb*exp(-(x-a)%2Fc)*x%2Fd
    beta = (plus_MW - eta)/alpha
    
    if LMW <= eta:
        return 0
    x = LMW
    a = eta
    b = alpha-1
    c = beta
    d = beta**alpha*gamma(alpha)
    return beta*alpha+eta-((c*(x - a)**b * (a* incomp_gamma(1 + b, ( x-a )/c) + c* incomp_gamma(2 + b, (-a + x)/c)))/(d* ((x-a)/c)**b))

In [3]:
# And now test the code with a simple Gamma distribution

eta = 90
alpha = 0.75
plus_MW = 165
# Define a list of LMW's
LMW = [90+(i+1)*14 for i in range(19)]

# Calculate the cumulative probability and mass weighted probability
cum_prob = [0]
cum_mass_prob = [0]
for M in LMW:
    cum_prob.append(Cum_pM(M, eta, alpha, plus_MW))
    cum_mass_prob.append(Cum_pMM(M, eta, alpha, plus_MW))
cum_prob.append(1)
cum_mass_prob.append(plus_MW)

# Calculate incremental molar fraction by subtracting previous values
Molar_frac = [cum_prob[i]-cum_prob[i-1] for i in range(1,len(cum_prob))]

# And average MW's in the LMW bins
Avg_MWs = [(cum_mass_prob[i]-cum_mass_prob[i-1])/Molar_frac[i-1] for i in range(1,len(cum_prob))]

sum_mass = 0
# Check total mole weighted masses add to target C7+ MW
for i in range(len(Molar_frac)):
    sum_mass += Molar_frac[i]*Avg_MWs[i]
    
print('Comp,  Mole Frac,   Avg MW,   Mass')
for i in range(len(Molar_frac)):
    print(i,'     ',round(Molar_frac[i],4),',  ',round(Avg_MWs[i],4),',  ', round(Molar_frac[i]*Avg_MWs[i],4) )

print(' ')
print('Total:   ',sum(Molar_frac),',         -       ,',sum_mass)

Comp,  Mole Frac,   Avg MW,   Mass
0       0.2347 ,   95.8264 ,   22.4935
1       0.138 ,   110.6368 ,   15.2695
2       0.1051 ,   124.7189 ,   13.1081
3       0.0839 ,   138.753 ,   11.639
4       0.0684 ,   152.7717 ,   10.456
5       0.0566 ,   166.7836 ,   9.435
6       0.0472 ,   180.7918 ,   8.5259
7       0.0396 ,   194.7978 ,   7.7045
8       0.0333 ,   208.8024 ,   6.9577
9       0.0282 ,   222.806 ,   6.2769
10       0.0239 ,   236.8089 ,   5.6562
11       0.0203 ,   250.8114 ,   5.0907
12       0.0173 ,   264.8134 ,   4.5762
13       0.0147 ,   278.8151 ,   4.1087
14       0.0126 ,   292.8166 ,   3.6848
15       0.0108 ,   306.8179 ,   3.301
16       0.0092 ,   320.8191 ,   2.9542
17       0.0079 ,   334.8201 ,   2.6411
18       0.0068 ,   348.821 ,   2.3591
19       0.0417 ,   450.4364 ,   18.762
 
Total:    1.0 ,         -       , 165.0
